In [1]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
! conda install -c rdkit rdkit -y
! pip install --pre deepchem

--2022-04-07 13:55:52--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   156MB/s    in 0.5s    

2022-04-07 13:55:53 (156 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6

In [5]:
import numpy as np
import deepchem as dc
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
import torch

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [3]:
def create_dataset(data_x, data_y):
  mols = [Chem.MolFromSmiles(smi) for smi in data_x]
  morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in mols]
  morgan_fps_array = np.asarray(morgan_fps, dtype=float)
  maccs_fps = [MACCSkeys.GenMACCSKeys(mol) for mol in mols]
  maccs_fps_array = np.asarray(maccs_fps, dtype=float)
  x = np.concatenate([morgan_fps_array, maccs_fps_array],axis=1)

  y = np.asarray(data_y, dtype=float).reshape(-1, 1)
  dataset = dc.data.NumpyDataset(X=x, y=y)
  return dataset

读取数据

In [6]:
data = pd.read_csv('chembl_acetylcholinesterase_bioactivity_data_processed.csv')
train_x, test_x, train_y, test_y = train_test_split(data['canonical_smiles'], data['pIC50'], test_size=0.2, random_state=1)

提取特征

In [7]:
train_dataset = create_dataset(train_x, train_y)
test_dataset = create_dataset(test_x, test_y)

建立模型

In [8]:
pytorch_model = torch.nn.Sequential(
        torch.nn.Linear(2215, 1000),
        torch.nn.ReLU(),
        torch.nn.Dropout(0.5),
        torch.nn.Linear(1000, 1))
model = dc.models.TorchModel(pytorch_model, dc.models.losses.L2Loss())

模型评估

In [9]:
res_list = []
for epoch in [5, 10, 15, 20, 25, 30, 40, 50]:
      model.fit(train_dataset, nb_epoch=epoch)

      train_predict = model.predict(train_dataset)    
      train_predict_list = train_predict.reshape(-1).tolist()
      train_y_list = train_dataset.y.tolist()

      train_mae = mean_absolute_error(train_predict_list, train_y_list)
      train_mse = mean_squared_error(train_predict_list, train_y_list)
      train_mape = mean_absolute_percentage_error(train_predict_list, train_y_list)
      train_r2 = r2_score(train_predict_list, train_y_list)

      test_predict = model.predict(test_dataset)
      test_predict_list = test_predict.reshape(-1).tolist()
      test_y_list = test_dataset.y.tolist()   

      test_mae = mean_absolute_error(test_predict_list, test_y_list)
      test_mse = mean_squared_error(test_predict_list, test_y_list)
      test_mape = mean_absolute_percentage_error(test_predict_list, test_y_list)
      test_r2 = r2_score(test_predict_list, test_y_list)

      keras_perf = {'mae':[train_mae, test_mae],
                'mse':[train_mse, test_mse],
                'mape':[train_mape, test_mape],
                'r2':[train_r2, test_r2]}
      res_list.append(pd.DataFrame.from_dict(keras_perf))
final_res = pd.concat(res_list, keys = [5, 10, 15, 20, 25, 30, 40, 50])

In [10]:
final_res

mae       mse      mape        r2
5  0  0.573497  0.573574  0.101198  0.641872
   1  0.693948  0.983184  0.130360  0.356647
10 0  0.401597  0.289727  0.070182  0.824873
   1  0.622774  0.821785  0.117809  0.499157
15 0  0.306614  0.183381  0.053042  0.895685
   1  0.621687  0.824351  0.116685  0.534493
20 0  0.284610  0.153899  0.050239  0.915789
   1  0.624801  0.835049  0.122021  0.534780
25 0  0.293601  0.153726  0.051477  0.912948
   1  0.642475  0.852755  0.125337  0.503929
30 0  0.210125  0.097197  0.035753  0.947770
   1  0.600515  0.803360  0.111317  0.558629
40 0  0.190653  0.084303  0.032648  0.957518
   1  0.591275  0.771047  0.109688  0.589050
50 0  0.181825  0.078682  0.031250  0.958896
   1  0.595505  0.773886  0.111247  0.576842